In [3]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import os
import re
import numpy as np

In [4]:
cur_path = Path(os.getcwd())
raw = cur_path / '..' / '..' / 'data' / '01_raw'

In [5]:
df = pd.read_csv(raw / 'sneaker.csv')
shoes = list(df.style_code)

In [6]:
options = Options()

In [7]:
#update driver here: http://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(executable_path= raw /'web-drivers'/ 'chromedriver')

In [8]:
driver.get("https://stockx.com/")

In [9]:
def get_random_ua():
    random_ua = ''
    ua_file = raw / 'web-drivers' / 'user-agents.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

In [ ]:
stockx = {}
long_delays = [i for i in range(10,21)]
delays = [i for i in range(3, 20)]

# for i in range(5):
for index, shoe in enumerate(shoes):
    user_agent = get_random_ua()
    options = Options()
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(chrome_options=options, executable_path= raw /'web-drivers'/ 'chromedriver')
    driver.get('https://www.spacejam.com/')

    if shoe in stockx.keys():
        continue

    try:
        time.sleep(np.random.choice(delays))
        url = 'https://stockx.com/search?s=' + str(shoe)
        driver.get(url)
        time.sleep(np.random.choice(long_delays))
        driver.find_element_by_css_selector('#main-content > div.browse-grid.loading.undefined > div.browse-grid > div:nth-child(1) > div > a > div.css-1spf6z8.e1b7sao0 > div.css-1iephdx.e1inh05x0').click()
        time.sleep(np.random.choice(delays))
        headers = {'user-agent': user_agent}
        soupMain = BeautifulSoup(driver.page_source, 'html.parser', headers=headers)
        resale = soupMain.find_all('div', class_='gauge-value')[2].text
        colorway = soupMain.find_all('span', {'data-testid': 'product-detail-colorway'})[0].text        

        stockx[shoe] = {'resale_price':resale, 'colorway':colorway}
        time.sleep(np.random.choice(delays))
    except:
        pass

In [ ]:
driver.quit()

In [ ]:
stockx

In [ ]:
with open(raw / 'stockx.json', 'w') as outfile:
    json.dump(stockx, outfile)